# Title: Hybrid Approach for Detecting Inflammatory Content on Social Media Using Traditional and Deep Learning Models

In [36]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.Collecting xgboost
  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/a2/48/d5da8591891327b0faf08179d420fba3893c6134bdd449497c5329e4cb01/xgboost-2.1.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 393.8 kB/s eta 0:05:18
   ---------------------------------------- 0.2/124.9 MB 1.4 MB/s eta 0:01:30
   ---------------------------------------- 0.4/124.9 MB 2.1 MB/s eta 0:01:00
   ---------------------------------------- 0.6/124.9 MB 2.9 MB/s eta 0:00:44
   ---------------------------------------- 0.8/124.9 MB 3.3 MB/s eta 0:00:38
   ---------------------------------------- 1.0/124.9 MB 3.5 MB/s eta 0:00:36
   ---------------------------------------- 1.3/124.9 MB 3.9 MB/s eta 0:00:32
   ------------------

In [55]:
import pandas as pd
import numpy as np
import re
import nltk
import xgboost as xgb
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Bidirectional, Flatten, Input, Reshape
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Flatten, Dense, Dropout
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [2]:
# Download required NLTK data
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
#load data
data=pd.read_csv('C:/Users/user/Downloads/PROJECT HATE SPEECH/data_huang_devansh.csv')

In [4]:
data.head()

,Content,Label
0,`- This is not ``creative``. Those are the di...,0
1,` :: the term ``standard model`` is itself le...,0
2,"True or false, the situation as of March 200...",0
3,"Next, maybe you could work on being less cond...",0
4,This page will need disambiguation.,0


In [5]:
# generating a new dataset by randomly extracting 10000 samples
datasample=data.sample(n=100000, random_state=42)
datasample

,Content,Label
668754,nope tabitha jai locascio idk who she is but...,0
761691,Angels now have 6 runs. Five of them have come...,0
52703,"wang, you are an idiot. The difference betwe...",1
569627,Silliness\n\nYou did something silly. Please d...,1
193945,I was doing amazing work and you know it. Sur...,0
...,...,...
388869,Either including on the list or creating a s...,0
143455,"wikipedia until April 7, 2007",0
772920,@user fucking cunt bitch,1
99241,"`, 3 July 2013 (UTC) ``I regret to inform you ...",0


In [6]:
#Step 3: Preprocessing
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def safe_stem(word, stemmer):
    try:
        return stemmer.stem(word)
    except RecursionError:
        return word

def preprocess_text(text):
    text = re.sub(r'\\xa0', ' ', str(text))
    text = re.sub(r'http\S+', '', text)
    text = ''.join([char for char in text if ord(char) < 128])
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = text.lower()
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]
    text = [safe_stem(word, stemmer) for word in text]
    return ' '.join(text)

data['clean_Content'] = data['Content'].apply(preprocess_text)

In [7]:
data.head()

,Content,Label,clean_Content
0,`- This is not ``creative``. Those are the di...,0,creativ dictionari definit term insur ensur pr...
1,` :: the term ``standard model`` is itself le...,0,term standard model less npov think wed prefer...
2,"True or false, the situation as of March 200...",0,true fals situat march saudi propos land peac ...
3,"Next, maybe you could work on being less cond...",0,next mayb could work less condescend suggest r...
4,This page will need disambiguation.,0,page need disambigu


In [8]:
# Subset data
subset_size = 10000
data_subset = data.sample(n=subset_size, random_state=42)

In [9]:
data_subset

,Content,Label,clean_Content
668754,nope tabitha jai locascio idk who she is but...,0,nope tabitha jai locascio idk fake use live id...
761691,Angels now have 6 runs. Five of them have come...,0,angel run five come courtesi mike trout homer ...
52703,"wang, you are an idiot. The difference betwe...",1,wang idiot differ year old year old intellig c...
569627,Silliness\n\nYou did something silly. Please d...,1,silli someth silli pleas dont silli jun utc
193945,I was doing amazing work and you know it. Sur...,0,amaz work know sure respons may sharp see half...
...,...,...,...
241197,==Apologies== I never got back to you prope...,0,apolog never got back properli wynton haydn fe...
253542,==I'm Glad I Removed My Fear Of The Orange B...,0,im glad remov fear orang bar thank birthday me...
49351,== My cock == YOU FAILED AGAIN TO BLOCK ME ...,1,cock fail block wikipedian ha ha nice big juic...
59326,"Hey, jack off. Are you the one who's got hi...",1,hey jack one who got head way wolf hudson ass


In [10]:
# Preprocess text
data_subset['clean_Content'] = data_subset['Content'].apply(preprocess_text)

In [11]:
# Separate the features and the target
X = data_subset['clean_Content']
y = data_subset['Label']

In [12]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
#TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train).toarray()
X_test_tfidf = tfidf_vectorizer.transform(X_test).toarray()

In [30]:
# Word2Vec Embeddings
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [31]:
# Define vocabulary size and maximum sequence length
vocab_size = len(tokenizer.word_index) + 1
maxlen = 100

In [32]:
# Padding sequences
X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

In [33]:
# Load or train a Word2Vec model
word2vec_model = Word2Vec(sentences=X_train, vector_size=100, window=5, min_count=1, workers=4)
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = word2vec_model.wv[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        continue

In [34]:
# Traditional Machine Learning Models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, solver='saga', random_state=42),
    "Naive Bayes": MultinomialNB(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Gradient Boosting Machine (GBM)": GradientBoostingClassifier(),
    "SVM": SVC()
}

In [35]:
# Traditional Machine Learning Models Predictions
for model_name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    predictions = model.predict(X_test_tfidf)
    print(f"{model_name} Classification Report:")
    print(classification_report(y_test, predictions))
    print(f"Number of Hate Speech : {np.sum(predictions == 1)}")
    print(f"Number of Non-Hate Speech : {np.sum(predictions == 0)}\n")

Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.99      0.93      1686
           1       0.76      0.20      0.32       314

    accuracy                           0.86      2000
   macro avg       0.82      0.60      0.62      2000
weighted avg       0.85      0.86      0.83      2000

Number of Hate Speech : 84
Number of Non-Hate Speech : 1916

Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      1686
           1       0.79      0.08      0.15       314

    accuracy                           0.85      2000
   macro avg       0.82      0.54      0.53      2000
weighted avg       0.84      0.85      0.80      2000

Number of Hate Speech : 33
Number of Non-Hate Speech : 1967

Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.92      0.91      1686


In [ ]:
# Step 9: Deep Learning Models

In [36]:
# CNN Model
cnn_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=maxlen, trainable=False),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

cnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

cnn_model.fit(X_train_pad, y_train, epochs=5, batch_size=64, validation_data=(X_test_pad, y_test))


C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8435 - loss: 0.5009 - val_accuracy: 0.8445 - val_loss: 0.4313
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8453 - loss: 0.4356 - val_accuracy: 0.8425 - val_loss: 0.4295
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8389 - loss: 0.4402 - val_accuracy: 0.8455 - val_loss: 0.4315
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8438 - loss: 0.4316 - val_accuracy: 0.8445 - val_loss: 0.4361
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.8478 - loss: 0.4237 - val_accuracy: 0.8425 - val_loss: 0.4423


In [49]:
cnn_pred = (cnn_model.predict(X_test_pad) > 0.5).astype("int32")
print("CNN Model Classification Report:")
print(classification_report(y_test, cnn_pred))
print(f"Number of Hate Speech : {np.sum(cnn_pred == 1)}")
print(f"Number of Non-Hate Speech : {np.sum(cnn_pred == 0)}\n")

63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step
CNN Model Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.99      0.91      1686
           1       0.48      0.03      0.06       314

    accuracy                           0.84      2000
   macro avg       0.66      0.51      0.49      2000
weighted avg       0.79      0.84      0.78      2000

Number of Hate Speech : 21
Number of Non-Hate Speech : 1979



In [51]:
# LSTM Model
lstm_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=False),
    Bidirectional(LSTM(100, return_sequences=True)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
lstm_model.fit(X_train_pad, y_train, epochs=5, batch_size=64, validation_data=(X_test_pad, y_test))

Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 88ms/step - accuracy: 0.8467 - loss: 0.4812 - val_accuracy: 0.8435 - val_loss: 0.4324
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 10s 80ms/step - accuracy: 0.8396 - loss: 0.4456 - val_accuracy: 0.8445 - val_loss: 0.4307
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 86ms/step - accuracy: 0.8518 - loss: 0.4215 - val_accuracy: 0.8440 - val_loss: 0.4369
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step - accuracy: 0.8424 - loss: 0.4386 - val_accuracy: 0.8430 - val_loss: 0.4309
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 11s 90ms/step - accuracy: 0.8466 - loss: 0.4357 - val_accuracy: 0.8415 - val_loss: 0.4325


In [52]:
# LSTM Prediction model
lstm_pred = (lstm_model.predict(X_test_pad) > 0.5).astype("int32")
print("LSTM Model Classification Report:")
print(classification_report(y_test, lstm_pred))
print(f"Number of Hate Speech : {np.sum(lstm_pred == 1)}")
print(f"Number of Non-Hate Speech : {np.sum(lstm_pred == 0)}\n")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
LSTM Model Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.99      0.91      1686
           1       0.44      0.04      0.06       314

    accuracy                           0.84      2000
   macro avg       0.64      0.51      0.49      2000
weighted avg       0.78      0.84      0.78      2000

Number of Hate Speech : 25
Number of Non-Hate Speech : 1975



In [56]:
# RNN Model
rnn_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], trainable=False),
    SimpleRNN(100, return_sequences=True),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

rnn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
rnn_model.fit(X_train_pad, y_train, epochs=5, batch_size=64, validation_data=(X_test_pad, y_test))

Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - accuracy: 0.8377 - loss: 0.4980 - val_accuracy: 0.8395 - val_loss: 0.4897
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.8439 - loss: 0.4450 - val_accuracy: 0.8435 - val_loss: 0.4437
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.8509 - loss: 0.4259 - val_accuracy: 0.8400 - val_loss: 0.4409
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.8434 - loss: 0.4422 - val_accuracy: 0.8420 - val_loss: 0.4447
Epoch 5/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.8488 - loss: 0.4268 - val_accuracy: 0.8410 - val_loss: 0.4507


In [57]:
# RNN Prediction model
rnn_pred = (rnn_model.predict(X_test_pad) > 0.5).astype("int32")
print("RNN Model Classification Report:")
print(classification_report(y_test, rnn_pred))
print(f"Number of Hate Speech : {np.sum(rnn_pred == 1)}")
print(f"Number of Non-Hate Speech : {np.sum(rnn_pred == 0)}\n")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step
RNN Model Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.99      0.91      1686
           1       0.36      0.02      0.03       314

    accuracy                           0.84      2000
   macro avg       0.60      0.51      0.47      2000
weighted avg       0.77      0.84      0.77      2000

Number of Hate Speech : 14
Number of Non-Hate Speech : 1986



In [ ]:
# Hybrid Model with CNN Feature Extraction
#  X_train, X_test, y_train, y_test are already defined

In [38]:
# Step 3: CNN Feature Extraction from Word2Vec embeddings
cnn_feature_model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=100, weights=[embedding_matrix], input_length=maxlen, trainable=False),
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Flatten()
])

cnn_feature_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

cnn_features_train = cnn_feature_model.predict(X_train_pad, batch_size=32)
cnn_features_test = cnn_feature_model.predict(X_test_pad, batch_size=32)

 20/250 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step   

C:\Users\user\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [39]:
# Flatten CNN features
cnn_features_train_flat = cnn_features_train.reshape(cnn_features_train.shape[0], -1)
cnn_features_test_flat = cnn_features_test.reshape(cnn_features_test.shape[0], -1)

In [40]:
# Step 4: Combining TF-IDF and CNN features
X_train_combined = np.hstack((X_train_tfidf, cnn_features_train_flat))
X_test_combined = np.hstack((X_test_tfidf, cnn_features_test_flat))

In [ ]:
#  Training and Evaluating hybrid Models

In [47]:
# XGBoost with CNN Feature Extraction from Word2Vec embeddings
print("Training XGBoost with combined features...")
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train_combined, y_train)

xgb_pred = xgb_model.predict(X_test_combined)
print("Hybrid XGBoost Model with Combined Features Classification Report:")
print(classification_report(y_test, xgb_pred))
print(f"Number of Hate Speech: {np.sum(xgb_pred == 1)}")
print(f"Number of Non-Hate Speech: {np.sum(xgb_pred == 0)}\n")

Training XGBoost with combined features...


C:\Users\user\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [16:18:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Hybrid XGBoost Model with Combined Features Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.97      0.92      1686
           1       0.66      0.29      0.41       314

    accuracy                           0.86      2000
   macro avg       0.77      0.63      0.66      2000
weighted avg       0.85      0.86      0.84      2000

Number of Hate Speech: 140
Number of Non-Hate Speech: 1860



In [48]:
# SVM CNN Feature Extraction from Word2Vec embeddings
print("Training SVM with combined features...")
svm_model = SVC(kernel='linear')
svm_model.fit(X_train_combined, y_train)

svm_pred = svm_model.predict(X_test_combined)
print("Hybrid SVM Model with Combined Features Classification Report:")
print(classification_report(y_test, svm_pred))
print(f"Number of Hate Speech: {np.sum(svm_pred == 1)}")
print(f"Number of Non-Hate Speech: {np.sum(svm_pred == 0)}\n")

Training SVM with combined features...
Hybrid SVM Model with Combined Features Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.97      0.93      1686
           1       0.72      0.36      0.48       314

    accuracy                           0.88      2000
   macro avg       0.81      0.67      0.71      2000
weighted avg       0.86      0.88      0.86      2000

Number of Hate Speech: 156
Number of Non-Hate Speech: 1844

